## hyperactive - unified interfaces for optimizers and experiments

### "experiment" = optimization problem

"experiment" classes model optimization functions and ML experiments under one API

Examples below:
1. simple objective function - parabola function
2. ML cross-validation experiment in sklearn

#### user defined objective function

simple objective definition:

* function with single dict argument
* keys are variable names
* function evaluates variables and returns float
* maximization is assumed later

In [ ]:
"""Hyperactive optimization library introduction notebook.

This notebook demonstrates unified interfaces for optimizers and experiments
using the Hyperactive optimization library.
"""


def sphere(opt):
    """Evaluate sphere function for optimization.

    Parameters
    ----------
    opt : dict
        Dictionary with 'x' and 'y' keys containing numeric values.

    Returns
    -------
    float
        Negative sum of squares (for maximization).
    """
    x = opt["x"]
    y = opt["y"]

    return -(x**2) - y**2

to evaluate:

In [2]:
sphere({"x": 2, "y": 3})

-13

#### parametric objective functions - parametric

parametric objective functions are classes:

* construct with parameters
* call `evaluate` with `dict`

`hyperactive` comes with predefined objective functions.

These are parametric through the constructor

Example: `Parabola` function, docstring outlines parametric form

In [3]:
from hyperactive.experiment.bench import Parabola

Parabola?

Init signature: Parabola(a=1.0, b=0.0, c=0.0)
Docstring:      Parabola class.
Init docstring: Construct BaseObject.
File:           c:\workspace\hyperactive\src\hyperactive\experiment\bench\_parabola.py
Type:           type
Subclasses:     

In [4]:
parabola = Parabola(a=42, b=3, c=4)

parametric objectives are evaluated via `evaluate` method:

In [5]:
score, metadata = parabola.evaluate({"x": 2, "y": 3})  # also returns metadata
score

np.float64(564.0)

instances of parametric objectives are also directly callable

In [6]:
parabola(x=2, y=3)
# output is always np.float64

np.float64(-564.0)

the "experiment" class has two sets of variables:

* optimization variables = inputs of the objective - inspectable via `paramnames`
* parameters of the experiment = constant in the objective
    * these are params of `__init__`
    * and are inspectable via `get_params`

In [7]:
parabola.paramnames()

['x', 'y']

In [8]:
list(parabola.get_params())

['a', 'b', 'c']

call via `score` method:

* returns two objects: the value, and metadata (in a dict)
* input is a single dict, the `**` variant of a direct call

In [9]:
parabola.score({"x": 2, "y": 3})

(np.float64(-564.0), {})

#### sklearn cross-validation

"experiment" can be more complicated - e.g., a cross-validation experiment

this is a single tuning-evaluation step for an `sklearn` estimator

In [ ]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC

from hyperactive.experiment.integrations import SklearnCvExperiment

X, y = load_iris(return_X_y=True)

sklearn_exp = SklearnCvExperiment(
    estimator=SVC(),
    scoring=accuracy_score,
    cv=KFold(n_splits=3, shuffle=True),
    X=X,
    y=y,
)

usage syntax same as for the simple parabola!

In [11]:
sklearn_exp(C=1, gamma=0.3)

np.float64(0.9733333333333333)

In [12]:
sklearn_exp.paramnames()

['C',
 'break_ties',
 'cache_size',
 'class_weight',
 'coef0',
 'decision_function_shape',
 'degree',
 'gamma',
 'kernel',
 'max_iter',
 'probability',
 'random_state',
 'shrinking',
 'tol',
 'verbose']

`get_params` works like in `sklearn` and is nested

note that similar parameters appear as in `paramnames`

* parameters in `paramnames` are optimized over
* parameters in `get_params` are default values, if not set in `score`

In [13]:
list(sklearn_exp.get_params().keys())

['X',
 'cv',
 'estimator',
 'scoring',
 'y',
 'estimator__C',
 'estimator__break_ties',
 'estimator__cache_size',
 'estimator__class_weight',
 'estimator__coef0',
 'estimator__decision_function_shape',
 'estimator__degree',
 'estimator__gamma',
 'estimator__kernel',
 'estimator__max_iter',
 'estimator__probability',
 'estimator__random_state',
 'estimator__shrinking',
 'estimator__tol',
 'estimator__verbose']

In [14]:
sklearn_exp.score({"C": 1, "gamma": 0.3})

(np.float64(0.98),
 {'score_time': array([0.        , 0.01113176, 0.00051761]),
  'fit_time': array([0.        , 0.        , 0.00100803]),
  'n_test_samples': 150})

### use of optimizers

#### optimizing a custom objective

In [ ]:
def sphere(opt):
    """Evaluate sphere function for optimization.

    Parameters
    ----------
    opt : dict
        Dictionary with 'x' and 'y' keys containing numeric values.

    Returns
    -------
    float
        Negative sum of squares (for maximization).
    """
    x = opt["x"]
    y = opt["y"]

    return -(x**2) - y**2

In [ ]:
import numpy as np

from hyperactive.opt import HillClimbing

hillclimbing_config = {
    "search_space": {
        "x": np.linspace(-10, 10, 100),
        "y": np.linspace(-10, 10, 100),
    },
    "n_iter": 1000,
}
hill_climbing = HillClimbing(**hillclimbing_config, experiment=sphere)

hill_climbing.solve()

#### Grid search & sklearn CV

In [ ]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC

from hyperactive.experiment.integrations import SklearnCvExperiment

X, y = load_iris(return_X_y=True)

sklearn_exp = SklearnCvExperiment(
    estimator=SVC(),
    scoring=accuracy_score,
    cv=KFold(n_splits=3, shuffle=True),
    X=X,
    y=y,
)

In [ ]:
from hyperactive.opt import GridSearchSk as GridSearch

param_grid = {
    "C": [0.01, 0.1, 1, 10],
    "gamma": [0.0001, 0.01, 0.1, 1, 10],
}
grid_search = GridSearch(param_grid=param_grid, experiment=sklearn_exp)

grid_search.solve()

#### hill climbing & sklearn CV

In [ ]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC

from hyperactive.experiment.integrations import SklearnCvExperiment

X, y = load_iris(return_X_y=True)

sklearn_exp = SklearnCvExperiment(
    estimator=SVC(),
    scoring=accuracy_score,
    cv=KFold(n_splits=3, shuffle=True),
    X=X,
    y=y,
)

In [ ]:
import numpy as np

from hyperactive.opt import HillClimbing

hillclimbing_config = {
    "search_space": {
        "C": np.array([0.01, 0.1, 1, 10]),
        "gamma": np.array([0.0001, 0.01, 0.1, 1, 10]),
    },
    "n_iter": 100,
}
hill_climbing = HillClimbing(**hillclimbing_config, experiment=sklearn_exp)

hill_climbing.solve()

### full sklearn integration as estimator

`OptCV` allows `sklearn` tuning via any tuning algorithm.

Below, we show tuning via:

* standard `GridSearch`
* `HillClimbing` from `gradient-free-optimizers`

##### `OptCV` tuning via `GridSearch`

In [ ]:
# 1. defining the tuned estimator
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from hyperactive.integrations.sklearn import OptCV
from hyperactive.opt import GridSearchSk as GridSearch

param_grid = {"kernel": ["linear", "rbf"], "C": [1, 10]}
tuned_svc = OptCV(SVC(), optimizer=GridSearch(param_grid))

# 2. fitting the tuned estimator = tuning the hyperparameters
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tuned_svc.fit(X_train, y_train)

# 3. making predictions with the tuned estimator
y_pred = tuned_svc.predict(X_test)

best parameters and best estimator can be returned

In [22]:
best_params = tuned_svc.best_params_

In [23]:
tuned_svc.best_estimator_

SVC(C=1)

##### `OptCV` tuning via `HillClimbing`

In [ ]:
# 1. defining the tuned estimator
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from hyperactive.integrations.sklearn import OptCV
from hyperactive.opt import HillClimbing

# picking the optimizer is the only part that changes!
hill_climbing_config = {
    "search_space": {
        "C": np.array([0.01, 0.1, 1, 10]),
        "gamma": np.array([0.0001, 0.01, 0.1, 1, 10]),
    },
    "n_iter": 100,
}
hill_climbing = HillClimbing(**hill_climbing_config)

tuned_svc = OptCV(SVC(), optimizer=hill_climbing)

# 2. fitting the tuned estimator = tuning the hyperparameters
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tuned_svc.fit(X_train, y_train)

# 3. making predictions with the tuned estimator
y_pred = tuned_svc.predict(X_test)

best parameters and best estimator - works as before!

In [25]:
tuned_svc.best_params_

{'C': np.float64(10.0), 'gamma': np.float64(0.1)}

In [26]:
tuned_svc.best_estimator_

SVC(C=np.float64(10.0), gamma=np.float64(0.1))